<a href="https://colab.research.google.com/github/shubhamitradas/bert_optimization_strategies/blob/main/multi_label_with_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers nvidia-ml-py3 --quiet

     |████████████████████████████████| 4.4 MB 35.0 MB/s 
     |████████████████████████████████| 6.6 MB 66.6 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 30.9 MB/s 


In [2]:
# Install Weights and Biases
!pip install wandb -q

# Import wandb
import wandb

# Login with your authentication key
wandb.login()

# setup wandb environment variables
#%env WANDB_ENTITY=your-username/your-team-name
%env WANDB_PROJECT=toxic_distillbert_tesla_t4_experiments

     |████████████████████████████████| 1.8 MB 33.1 MB/s 
     |████████████████████████████████| 181 kB 64.2 MB/s 
     |████████████████████████████████| 145 kB 69.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=toxic_distillbert_tesla_t4_experiments


In [3]:
# Code for TPU packages install
#!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [4]:
# Preparing for TPU usage
#import torch_xla
#import torch_xla.core.xla_model as xm
#device = xm.xla_device()

In [5]:
import transformers
print(f"Transformers package version: {transformers.__version__}")

Transformers package version: 4.20.0


In [6]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [7]:
# Import all libraries
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm

# Huggingface transformers
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification,AutoTokenizer,AutoModel,DataCollatorWithPadding
from transformers import AlbertTokenizer, AlbertModel


import torch
import random,gc
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler



from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [8]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [9]:
checkPoint = 'distilbert-base-uncased' 

In [10]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/toxic_train.csv.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

Mounted at /content/drive/


In [11]:
import pandas as pd
df_full = pd.read_csv('/content/train.csv')
df_full.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split


texts = list(df_full["comment_text"])
label_names = df_full.drop(["id", "comment_text"], axis=1).columns
labels = df_full[label_names].values

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)



In [13]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

MAX_LENGTH = 200
BATCH_SIZE = 128
LEARNING_RATE = 1e-05

MODEL_NAME =  checkPoint
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) 


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [14]:
print_gpu_utilization()

GPU memory occupied: 0 MB.


In [15]:
class MultiLabelDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_len):

        self.encoded_inputs = tokenizer(texts, truncation=True, padding=True, 
            max_length=max_len, return_tensors="pt")
        self.labels = torch.tensor(labels, dtype=torch.float)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        inputs_ands_labels = dict()
        inputs_ands_labels['input_ids'] = self.encoded_inputs['input_ids'][index]  
        inputs_ands_labels['attention_mask'] = self.encoded_inputs['attention_mask'][index] 
        inputs_ands_labels['token_type_ids'] = None 
        inputs_ands_labels['labels'] = self.labels[index]
        return inputs_ands_labels


train_dataset = MultiLabelDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
test_dataset = MultiLabelDataset(test_texts, test_labels, tokenizer, MAX_LENGTH)
      
    

In [16]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [17]:
# Creating the customized model, by adding a drop out and a dense layer on top of distill bert to get the final output for the model. 
from torch.nn.functional import binary_cross_entropy_with_logits

class FineTunedBERTClass(torch.nn.Module):
    def __init__(self):
        super(FineTunedBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(MODEL_NAME)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 6)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)

        if labels is None:
            return output
        else:
            return binary_cross_entropy_with_logits(output, labels), output



model = FineTunedBERTClass()

model.to(device)   




Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FineTunedBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_fe

In [18]:
print_gpu_utilization()

GPU memory occupied: 1698 MB.


In [19]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

# **Vanilla Training.**

In [20]:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='tmp',
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    run_name="vanilla_setting",
    report_to="wandb",
    seed=seed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
   
)
result = trainer.train()
print_summary(result)
wandb.log(result.metrics)
wandb.finish()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 127656
  Num Epochs = 1
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 998
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: shubhamitra_das. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.143900,0.037922,0.792577,0.878755,0.928529


Saving model checkpoint to tmp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 31915
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




Time: 2111.51
Samples/second: 60.46
GPU memory occupied: 14054 MB.


epoch,▁
eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/roc_auc,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
total_flos,▁
train/epoch,▁██
train/global_step,▁███


In [21]:
print_gpu_utilization()
torch.cuda.empty_cache() 
gc.collect()
print_gpu_utilization()

GPU memory occupied: 14054 MB.
GPU memory occupied: 3134 MB.


## **Gradient Accumulation**

In [22]:
from transformers import Trainer, TrainingArguments

del model
model = FineTunedBERTClass()
model.to(device) 

training_args = TrainingArguments(
    output_dir='tmp',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    group_by_length = True,
    dataloader_num_workers =4,
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    run_name="gradient_accumulation",
    report_to="wandb",
    seed=seed
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
   
)
result = trainer.train()
print_summary(result)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.20.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
0,0.141900,0.038791,0.785098,0.870158,0.927150


Saving model checkpoint to tmp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
***** Running Evaluation *****
  Num examples = 31915
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




Time: 2178.65
Samples/second: 58.59
GPU memory occupied: 7196 MB.


In [23]:
wandb.log(result.metrics)
wandb.finish()

epoch,▁
eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/roc_auc,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
total_flos,▁
train/epoch,▁██
train/global_step,▁███


In [24]:
print_gpu_utilization()
torch.cuda.empty_cache() 
gc.collect()
print_gpu_utilization()

GPU memory occupied: 7196 MB.
GPU memory occupied: 3598 MB.


## **fp16**

In [25]:
from transformers import Trainer, TrainingArguments


del model
model = FineTunedBERTClass()
model.to(device) 

training_args = TrainingArguments(
    output_dir='tmp',
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    group_by_length = True,
    #dataloader_num_workers =4,
    fp16=True,
    run_name="fp16",
    report_to="wandb",
    seed=123
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
   
)
result = trainer.train()
print_summary(result)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.20.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.141800,0.038760,0.787651,0.874666,0.928153


Saving model checkpoint to tmp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 31915
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




Time: 688.03
Samples/second: 185.54
GPU memory occupied: 11222 MB.


In [26]:
wandb.log(result.metrics)
wandb.finish()

epoch,▁
eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/roc_auc,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
total_flos,▁
train/epoch,▁██
train/global_step,▁███


In [27]:
print_gpu_utilization()
torch.cuda.empty_cache() 
gc.collect()
print_gpu_utilization()

GPU memory occupied: 11222 MB.
GPU memory occupied: 5338 MB.


## **AdaFactor**

In [28]:
from transformers import Trainer, TrainingArguments


del model
model = FineTunedBERTClass()
model.to(device) 

training_args = TrainingArguments(
    output_dir='tmp',
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    group_by_length = True,
    optim="adafactor",
    report_to="wandb",
    run_name="adafactor",
    dataloader_num_workers =4,
    seed=123
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
   
)
result = trainer.train()
print_summary(result)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.20.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.150800,0.038035,0.790175,0.874451,0.928529


Saving model checkpoint to tmp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
***** Running Evaluation *****
  Num examples = 31915
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




Time: 2121.59
Samples/second: 60.17
GPU memory occupied: 14662 MB.


In [29]:
print_gpu_utilization()
torch.cuda.empty_cache() 
gc.collect()
print_gpu_utilization()

GPU memory occupied: 14662 MB.
GPU memory occupied: 4958 MB.


## **8-bit Adam**

In [30]:
!pip install bitsandbytes-cuda112 -q

     |████████████████████████████████| 4.2 MB 32.0 MB/s 


In [ ]:
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names

from transformers import Trainer, TrainingArguments


del model
model = FineTunedBERTClass()
model.to(device) 


training_args = TrainingArguments(
    output_dir='tmp',
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    evaluation_strategy="epoch",
    group_by_length = True,
    dataloader_num_workers =4,
    run_name="8-bit-adam",
    report_to="wandb",
    seed=123
)

decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    optimizers=(adam_bnb_optim, None),
    compute_metrics=compute_metrics,
   
)
result = trainer.train()
print_summary(result)
wandb.log(result.metrics)
wandb.finish()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.20.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

Epoch,Training Loss,Validation Loss


Saving model checkpoint to tmp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


In [ ]:
print_gpu_utilization()
torch.cuda.empty_cache() 
gc.collect()
print_gpu_utilization()

## **Full Settings**

In [33]:
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names

from transformers import Trainer, TrainingArguments


del model
model = FineTunedBERTClass()
model.to(device) 


training_args = TrainingArguments(
    output_dir='tmp',
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    evaluation_strategy="epoch",
    group_by_length = True,
    dataloader_num_workers =4,
    gradient_accumulation_steps=4,
    fp16=True,
    run_name="full_settings",
    report_to="wandb",
    seed=123
)


decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    optimizers=(adam_bnb_optim, None),
    compute_metrics=compute_metrics,
   
)
result = trainer.train()
print_summary(result)
wandb.log(result.metrics)
wandb.finish()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.20.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
0,No log,0.051696,0.741946,0.825083,0.923108


***** Running Evaluation *****
  Num examples = 31915
  Batch size = 128


Training completed. Do not forget to share your model on huggingface.co/models =)




Time: 658.92
Samples/second: 193.73
GPU memory occupied: 10650 MB.


epoch,▁
eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/roc_auc,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
total_flos,▁
train/epoch,▁▁
train/global_step,▁▁▁


In [34]:
!nvidia-smi

Mon Jun 20 09:42:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |  10650MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------